In [ ]:
import pandas as pd

df = pd.read_csv('Reliance.csv')

,Date,Open,High,Low,Close,Volume,Remarks,Sector
0,2021-07-01,2198.65,2233.53,2157.69,2193.63,1453571.0,Solid trading,Energy
1,2021-07-02,2323.36,2371.23,2308.90,2346.12,1222924.0,Good start,Energy
2,2021-07-05,2245.37,2290.75,2203.67,2248.41,1442326.0,Steady,Energy
3,2021-07-06,2122.82,2144.80,2077.30,2114.97,1351019.0,Steady,Energy
4,2021-07-07,2334.97,2362.93,2321.43,2345.10,1351019.0,Steady,Energy
5,2021-07-08,2072.37,2097.22,2034.75,2065.81,1186676.0,Solid trading,Energy
6,2021-07-09,2306.10,2353.60,2295.42,2320.88,1352233.0,Good start,Energy
7,2021-07-12,2141.05,2187.07,2100.91,2152.09,1477806.0,Steady,Energy
8,2021-07-13,2396.86,2441.08,2365.27,2408.20,1437627.0,Steady,Energy
9,2021-07-14,2396.86,2139.35,2093.86,2114.53,1071523.0,Steady,Energy
